# Applied Machine Learning
## Grid search to determine the best training parameters
- Author: Lorien Pratt
- Copyright: Quantellia LLC 2019.  All Rights Reserved

Grid search does many model runs to find which one produces the best result after a few epochs (assuming that this is a good proxy for the final learning performance, which may or may not be true).

Grid search explores multiple network architectures (number of layers, number of hidden units per layer) and other learning parameters. 

## Setup

In [1]:
# Set up to be able to invoke R from inside this Python 2 notebook
#%load_ext rpy2.ipython
#import rpy2.rinterface

##### Install and initialize the H2O library, which we will use to do the grid search
Note that this will generate a lot of warnings. These are expected, and not errors but rather notifications

In [2]:
require(h2o)
h2o.init()
h2o.no_progress() # Turns off progress bars, which don't display well in Jupyter

Loading required package: h2o

----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit http://docs.h2o.ai

----------------------------------------------------------------------


Attaching package: ‘h2o’

The following objects are masked from ‘package:stats’:

    cor, sd, var

The following objects are masked from ‘package:base’:

    &&, %*%, %in%, ||, apply, as.factor, as.numeric, colnames,
    colnames<-, ifelse, is.character, is.factor, is.numeric, log,
    log10, log1p, log2, round, signif, trunc



 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         8 minutes 37 seconds 
    H2O cluster timezone:       Etc/UTC 
    H2O data parsing timezone:  UTC 
    H2O cluster version:        3.26.0.10 
    H2O cluster version age:    6 days  
    H2O cluster name:           H2O_started_from_R_jupyter_ldd243 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   0.38 GB 
    H2O cluster total cores:    1 
    H2O cluster allowed cores:  1 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    H2O Internal Security:      FALSE 
    H2O API Extensions:         Amazon S3, XGBoost, Algos, AutoML, Core V3, TargetEncoder, Core V4 
    R Version:                  R version 3.6.1 (2019-07-05) 



Set up my initials for file names

In [4]:
my_initials<-"jing"

Read in the test and training files that we created in the Prepare Data lesson, and convert them to h2o's internal "hex" format

In [5]:
train_filename<-paste0("data/",my_initials,"_train_auto.csv"); print( train_filename )
test_filename<-paste0("data/",my_initials,"_test_auto.csv"); print( test_filename )
backtest_filename<-paste0("data/",my_initials,"_backtest_auto.csv"); print( backtest_filename )

[1] "data/jing_train_auto.csv"
[1] "data/jing_test_auto.csv"
[1] "data/jing_backtest_auto.csv"


Read in the test and training files you created in the previous step. Convert them to h2o files along the way.

In [6]:
train_hex <- h2o.importFile(train_filename, parse = TRUE, header = TRUE, 
                            sep = "", col.names = NULL, col.types = NULL, na.strings = NULL)
test_hex <- h2o.importFile(test_filename, parse = TRUE, header = TRUE, 
                           sep = "", col.names = NULL, col.types = NULL, na.strings = NULL)

Tell the grid search which of the columns are predictors.  First, let's look at the top of the dataset again to remind us of the structure...

In [7]:
head(train_hex)

sepal_length,sepal_width,petal_length,petal_width,class
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4.5,2.3,1.3,0.3,1
6.3,2.7,4.9,1.8,0
6.7,3.3,5.7,2.5,0
6.4,2.8,5.6,2.1,0
6.5,3.0,5.2,2.0,0
4.7,3.2,1.3,0.2,1


Set the predictor columns and chec that they're the right ones

In [8]:
predictors <- c(1,2,3,4,5)
names(train_hex)[predictors]

[1] "sepal_length" "sepal_width"  "petal_length" "petal_width"  "class"

Tell the model training which of the columns is the target column (in this case, the very last column, mpg)

In [9]:
targetcol<-ncol(train_hex)

Tell the grid search which of the columns are predictors.  First, let's loo at the top of the dataset again to remind us of the structure...

Create a set of grid search *hyperparameters* .  These are the alternative structures we'll try to see which one
creates the best results after running the speciied number of epochs

In [10]:
hyper_params <- list(
    hidden=list(1, 5, 10, c(5,5), c(10,10,10)),
    l1=c(0, .01, .00001),
    l2=c(0, .01, 0.001, .00001),
    input_dropout_ratio=c(0, .01, .0001),
    epochs=c(100)
)

Run the grid test with these parameters.  This can take a little while, during which there will be no feedback.

In [11]:
grid_result <- h2o.grid(
    algorithm="deeplearning",
    x=predictors,
    y=targetcol,
    grid_id="grid_1", # Can't be reused; consider incrementing on subsequent runs. TBD: try kernel restart for this instead
    training_frame=train_hex,
    validation_frame=test_hex,
    quiet_mode=FALSE,
    export_weights_and_biases=TRUE,
    activation="Tanh",
    autoencoder=FALSE,
    ignore_const_cols=FALSE,
    standardize=FALSE,
    train_samples_per_iteration=0,
    adaptive_rate=FALSE, # Manaully tuned learning rate
    classification_stop = -1, # Dispable automatic stopping
    regression_stop = -1, # Disable automatic stopping
    stopping_rounds = 0, # Don't stop automatically
    hyper_params = hyper_params

)

Warning message in .verify_dataxy(params$training_frame, x, y):
“removing response variable from the explanatory variables”

Display the grid search results

In [12]:
h2o.getGrid("grid_1", sort_by="mse", decreasing=FALSE)

H2O Grid Details

Grid ID: grid_1 
Used hyper parameters: 
  -  epochs 
  -  hidden 
  -  input_dropout_ratio 
  -  l1 
  -  l2 
Number of models: 180 
Number of failed models: 0 

Hyper-Parameter Search Summary: ordered by increasing mse
  epochs       hidden input_dropout_ratio     l1     l2        model_ids
1  100.0       [5, 5]                0.01 1.0E-5 1.0E-5 grid_1_model_174
2  100.0 [10, 10, 10]              1.0E-4    0.0  0.001 grid_1_model_105
3  100.0 [10, 10, 10]              1.0E-4    0.0 1.0E-5 grid_1_model_150
4  100.0 [10, 10, 10]                 0.0 1.0E-5 1.0E-5 grid_1_model_170
5  100.0 [10, 10, 10]                0.01    0.0    0.0  grid_1_model_10
                    mse
1  9.809676176837586E-4
2 0.0010111315694325115
3 0.0011047155141876667
4  0.001187502401652319
5 0.0012007269888056446

---
    epochs hidden input_dropout_ratio     l1     l2        model_ids
175  100.0    [1]              1.0E-4 1.0E-5  0.001 grid_1_model_131
176  100.0    [1]              1.0E-